<a href="https://colab.research.google.com/github/GalinaChugreeva/ComENet/blob/main/ComENet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# print(torch.__version__)

In [3]:
!pip install dive-into-graphs
!pip install dive-into-graphs
!pip install torch_geometric
!pip install torch_sparse
!pip install torch_cluster
!pip install torch_scatter

  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch_sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1071448 sha256=bca4382a1676d9421ed7d45f941769d2cd42d6e3e633c2e8590c97c1d16e2807
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch_sparse
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch_cluster: filename=torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl size=714413 sha256=3835d4d89a8ba8fe91b6bfa9a62e0be7cba253fc9822f8830a2c6e1d5061995b
  Stored in directory: /root/.cache/pip/wheels/51/78/c3/536637b3cdcc3313aa5e8851a6c72b97f6a01877e68c7595e3
Successfully built torch_cluster
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1

In [4]:

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import tqdm
import re
from torch_geometric.data import Data, DataLoader
from torch_geometric.data import InMemoryDataset, download_url
from sklearn.utils import shuffle
import os.path as osp
from torch_scatter import scatter
from dig.threedgraph.dataset import QM93D
from dig.threedgraph.method import ComENet
from dig.threedgraph.evaluation import ThreeDEvaluator
from dig.threedgraph.method import run

In [5]:
from dig.version import __version__
print(__version__)

1.1.0


In [14]:
with open('/content/data_all_new.txt', 'r') as f:
    data = np.array(f.readlines()[1:])
# data

# df = {'idx': [], 'molecule': [], 'energy': [], 'number': []}
df = {'idx': [], 'number': [], 'energy': [], 'atom': [], 'molecule_struct': []}
i = 0
for j, x in enumerate((data)):
    if re.search(r'"\d{1,}', x):
        i = j + 2
        df['idx'].append(x[1:-1])
    elif '"' in x:
        last = data[j].split('\t')
        df1 = np.concatenate([data[i:j], [last[0]]], axis=None)
        df1 = pd.DataFrame([arr[:-1].split(' ') for arr in df1])
        df['energy'].append(float(last[1]))
        df['number'].append(last[2])
        df['atom'].append(np.array(df1.iloc[:, 0]))
        df['molecule_struct'].append(np.array(df1.iloc[:, 1:].transform(pd.to_numeric, errors='coerce').values.tolist()))
        # df['molecule'].append(df1)
molec_data = pd.DataFrame(df)

atom_ser = molec_data['atom']
for i in range(len(atom_ser)):
    for k in range(len(molec_data['atom'][i])):
        if molec_data['atom'][i][k]=='H':
            atom_ser[i][k]=1
        if molec_data['atom'][i][k]=='C':
            atom_ser[i][k]=6



data = pd.DataFrame(molec_data['energy'])

data = data.rename(columns={'energy': 'U0'})

target = {}
for name in ['mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U', 'H', 'G', 'Cv']:
    target[name] = np.expand_dims(np.zeros(len(data['U0'])),axis=-1)
target['U0'] = np.expand_dims(data['U0'],axis=-1)

data_list = []
for i in (range(len(molec_data['atom']))):
    R_i = torch.tensor(molec_data['molecule_struct'][i],dtype=torch.float32)
    z_i = torch.tensor(np.array(atom_ser[i], dtype='float32'),dtype=torch.int64)
    y_i = [torch.tensor(target[name][i],dtype=torch.float32) for name in ['mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve','U0', 'U', 'H', 'G', 'Cv']]
    data = Data(pos=R_i, z=z_i, y=y_i[7], mu=y_i[0], alpha=y_i[1], homo=y_i[2], lumo=y_i[3], gap=y_i[4], r2=y_i[5], zpve=y_i[6], U0=y_i[7], U=y_i[8], H=y_i[9], G=y_i[10], Cv=y_i[11])

    data_list.append(data)

In [1]:
# for step, batch_data in enumerate(DataLoader(data_list, batch_size=32, shuffle=True)):
#     print(step, batch_data.y)

In [24]:
# # Load the dataset and split
# dataset = QM93D(root='dataset/')
# target = 'U0'
# dataset.data.y = dataset.data[target]
# split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=110000, valid_size=10000, seed=42)
# train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]


In [25]:
def get_idx_split(data_size, train_size, valid_size, seed):
    ids = shuffle(range(data_size), random_state=seed)
    train_idx, val_idx, test_idx = torch.tensor(ids[:train_size]), torch.tensor(ids[train_size:train_size + valid_size]), torch.tensor(ids[train_size + valid_size:])
    split_dict = {'train':train_idx, 'valid':val_idx, 'test':test_idx}
    return split_dict

In [26]:
split_idx = get_idx_split(data_size=len(data_list), train_size=int(len(data_list)*0.7), valid_size=int(len(data_list)*0.2), seed=42)

In [27]:
# разбивка на датасеты

train_dataset = []
for i in split_idx['train']:
  train_dataset.append(data_list[i])


valid_dataset = []
for i in split_idx['valid']:
  valid_dataset.append(data_list[i])

test_dataset = []
for i in split_idx['test']:
  test_dataset.append(data_list[i])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [29]:
# Define model, loss, and evaluation

device = torch.device("cpu")


model = ComENet(#energy_and_force=False,
                cutoff=5.0, num_layers=4,
                  hidden_channels=128, out_channels=1,
                #int_emb_size=64,
                 # basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
                 # num_spherical=3, num_radial=6, envelope_exponent=5,
                #  num_before_skip=1, num_after_skip=2,
                num_output_layers=3)
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()


In [30]:
import time
import os
import torch
from torch.optim import Adam
from torch_geometric.data import DataLoader
import numpy as np
from torch.autograd import grad
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

class run():
    r"""
    The base script for running different 3DGN methods.
    """
    def __init__(self):
        pass

    def run(self, device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=500, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=50, weight_decay=0,
        energy_and_force=False, p=100, save_dir='', log_dir=''):
        r"""
        The run script for training and validation.

        Args:
            device (torch.device): Device for computation.
            train_dataset: Training data.
            valid_dataset: Validation data.
            test_dataset: Test data.
            model: Which 3DGN model to use. Should be one of the SchNet, DimeNetPP, and SphereNet.
            loss_func (function): The used loss funtion for training.
            evaluation (function): The evaluation function.
            epochs (int, optinal): Number of total training epochs. (default: :obj:`500`)
            batch_size (int, optinal): Number of samples in each minibatch in training. (default: :obj:`32`)
            vt_batch_size (int, optinal): Number of samples in each minibatch in validation/testing. (default: :obj:`32`)
            lr (float, optinal): Initial learning rate. (default: :obj:`0.0005`)
            lr_decay_factor (float, optinal): Learning rate decay factor. (default: :obj:`0.5`)
            lr_decay_step_size (int, optinal): epochs at which lr_initial <- lr_initial * lr_decay_factor. (default: :obj:`50`)
            weight_decay (float, optinal): weight decay factor at the regularization term. (default: :obj:`0`)
            energy_and_force (bool, optional): If set to :obj:`True`, will predict energy and take the minus derivative of the energy with respect to the atomic positions as predicted forces. (default: :obj:`False`)
            p (int, optinal): The forces’ weight for a joint loss of forces and conserved energy during training. (default: :obj:`100`)
            save_dir (str, optinal): The path to save trained models. If set to :obj:`''`, will not save the model. (default: :obj:`''`)
            log_dir (str, optinal): The path to save log files. If set to :obj:`''`, will not save the log files. (default: :obj:`''`)

        """

        model = model.to(device)
        num_params = sum(p.numel() for p in model.parameters())
        print(f'#Params: {num_params}')
        optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = StepLR(optimizer, step_size=lr_decay_step_size, gamma=lr_decay_factor)

        train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
        valid_loader = DataLoader(valid_dataset, vt_batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, vt_batch_size, shuffle=False)
        best_valid = float('inf')
        best_test = float('inf')

        if save_dir != '':
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
        if log_dir != '':
            if not os.path.exists(log_dir):
                os.makedirs(log_dir)
            writer = SummaryWriter(log_dir=log_dir)


        train_mae_arr=[]
        val_mae_arr=[]
        test_mae_arr=[]

        for epoch in range(1, epochs + 1):
            print("\n=====Epoch {}".format(epoch), flush=True)

            print('\nTraining...', flush=True)
            train_mae = self.train(model, optimizer, train_loader, energy_and_force, p, loss_func, device)
            train_mae_arr.append(train_mae)

            print('\n\nEvaluating...', flush=True)
            valid_mae = self.val(model, valid_loader, energy_and_force, p, evaluation, device)
            val_mae_arr.append(valid_mae)

            print('\n\nTesting...', flush=True)
            test_mae = self.val(model, test_loader, energy_and_force, p, evaluation, device)
            test_mae_arr.append(test_mae)

            print()
            print('train_mae_arr', train_mae_arr, 'val_mae_arr', val_mae_arr, 'test_mae_arr', test_mae_arr)
            print({'Train': train_mae, 'Validation': valid_mae, 'Test': test_mae})

            if log_dir != '':
                writer.add_scalar('train_mae', train_mae, epoch)
                writer.add_scalar('valid_mae', valid_mae, epoch)
                writer.add_scalar('test_mae', test_mae, epoch)

            if valid_mae < best_valid:
                best_valid = valid_mae
                best_test = test_mae
                if save_dir != '':
                    print('Saving checkpoint...')
                    checkpoint = {'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'scheduler_state_dict': scheduler.state_dict(), 'best_valid_mae': best_valid, 'num_params': num_params}
                    torch.save(checkpoint, os.path.join(save_dir, 'valid_checkpoint.pt'))

            scheduler.step()

        print(f'Best validation MAE so far: {best_valid}')
        print(f'Test MAE when got best validation result: {best_test}')

        if log_dir != '':
            writer.close()


    def train(self, model, optimizer, train_loader, energy_and_force, p, loss_func, device):
        r"""
        The script for training.

        Args:
            model: Which 3DGN model to use. Should be one of the SchNet, DimeNetPP, and SphereNet.
            optimizer (Optimizer): Pytorch optimizer for trainable parameters in training.
            train_loader (Dataloader): Dataloader for training.
            energy_and_force (bool, optional): If set to :obj:`True`, will predict energy and take the minus derivative of the energy with respect to the atomic positions as predicted forces. (default: :obj:`False`)
            p (int, optinal): The forces’ weight for a joint loss of forces and conserved energy during training. (default: :obj:`100`)
            loss_func (function): The used loss funtion for training.
            device (torch.device): The device where the model is deployed.

        :rtype: Traning loss. ( :obj:`mae`)

        """
        model.train()
        loss_accum = 0
        for step, batch_data in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            batch_data = batch_data.to(device)
            out = model(batch_data)
            if energy_and_force:
                force = -grad(outputs=out, inputs=batch_data.pos, grad_outputs=torch.ones_like(out),create_graph=True,retain_graph=True)[0]
                e_loss = loss_func(out, batch_data.y.unsqueeze(1))
                f_loss = loss_func(force, batch_data.force)
                loss = e_loss + p * f_loss
            else:
                loss = loss_func(out, batch_data.y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            loss_accum += loss.detach().cpu().item()
        return loss_accum / (step + 1)


    def val(self, model, data_loader, energy_and_force, p, evaluation, device):
        r"""
        The script for validation/test.

        Args:
            model: Which 3DGN model to use. Should be one of the SchNet, DimeNetPP, and SphereNet.
            data_loader (Dataloader): Dataloader for validation or test.
            energy_and_force (bool, optional): If set to :obj:`True`, will predict energy and take the minus derivative of the energy with respect to the atomic positions as predicted forces. (default: :obj:`False`)
            p (int, optinal): The forces’ weight for a joint loss of forces and conserved energy. (default: :obj:`100`)
            evaluation (function): The used funtion for evaluation.
            device (torch.device, optional): The device where the model is deployed.

        :rtype: Evaluation result. ( :obj:`mae`)

        """
        model.eval()

        preds = torch.Tensor([]).to(device)
        targets = torch.Tensor([]).to(device)

        if energy_and_force:
            preds_force = torch.Tensor([]).to(device)
            targets_force = torch.Tensor([]).to(device)

        for step, batch_data in enumerate(tqdm(data_loader)):
            batch_data = batch_data.to(device)
            out = model(batch_data)
            if energy_and_force:
                force = -grad(outputs=out, inputs=batch_data.pos, grad_outputs=torch.ones_like(out),create_graph=True,retain_graph=True)[0]
                preds_force = torch.cat([preds_force,force.detach_()], dim=0)
                targets_force = torch.cat([targets_force,batch_data.force], dim=0)
            preds = torch.cat([preds, out.detach_()], dim=0)
            targets = torch.cat([targets, batch_data.y.unsqueeze(1)], dim=0)

        input_dict = {"y_true": targets, "y_pred": preds}

        if energy_and_force:
            input_dict_force = {"y_true": targets_force, "y_pred": preds_force}
            energy_mae = evaluation.eval(input_dict)['mae']
            force_mae = evaluation.eval(input_dict_force)['mae']
            print({'Energy MAE': energy_mae, 'Force MAE': force_mae})
            return energy_mae + p * force_mae

        return evaluation.eval(input_dict)['mae']

In [ ]:
# Train and evaluate
run3d = run()
run3d.run(device,
          train_dataset=train_dataset,
          valid_dataset=valid_dataset,
          test_dataset=test_dataset,
          model=model,
          loss_func=loss_func, evaluation=evaluation,
          epochs=100, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15)

#Params: 990593

=====Epoch 1

Training...


  0%|          | 0/2678 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/dig/threedgraph/method/comenet/comenet.py:366: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:63.)
  b = torch.cross(-pos_ji, pos_in0).norm(dim=-1)
100%|██████████| 2678/2678 [11:00<00:00,  4.06it/s]



Evaluating...



100%|██████████| 765/765 [01:13<00:00, 10.44it/s]



Testing...


...


100%|██████████| 765/765 [01:14<00:00, 10.29it/s]



Testing...



100%|██████████| 383/383 [00:36<00:00, 10.49it/s]


train_mae_arr [4.055204213778415, 1.8628292446976904, 1.6316772502602883, 1.395118924999878, 1.227341976603792, 1.1519817577715326, 1.0612156420523235, 1.068930999390486, 1.0061492864724146, 0.90764278373761, 0.7819268785367002, 0.778746536367415, 0.802631254246023, 0.7386391211838043, 0.6157286095921899, 0.296317524211989, 0.28439199319074543, 0.3222034221446897, 0.29986566701337775, 0.3080198318767761, 0.2823844312108035, 0.26111988125673596, 0.2713740493398357, 0.26996878316991807, 0.25800652366863247, 0.2555891345738474, 0.260896972883451, 0.2767521185693321, 0.2590694594863994, 0.2598164523155321, 0.11204648507540577, 0.10730591115887438, 0.11298104967084548, 0.11079099785489231, 0.11203619078792147, 0.11312533609007079, 0.11393462890471158, 0.1127115717746145, 0.11786370556954576, 0.11314425235189904, 0.11270767134281127, 0.1085059194977912] val_mae_arr [5.495244026184082, 3.167360782623291, 2.6051743030548096, 0.5303632616996765, 0.2743622064590454, 2.0302789211273193, 3.139362


 26%|██▌       | 693/2678 [03:38<09:22,  3.53it/s]